In [1]:
import os
from typing import Optional
from openai import OpenAI
from dotenv import load_dotenv
from abc import ABC, abstractmethod
import json
import re
from exec_code import execute_code, extract_code_from_string
from prompt import CODE_DEVELOPER_SYSTEMPROMPT, DOCUMENTATION_DEVELOPER_SYSTEMPROMPT, EXECUTE_UIDESIGN_SYSTEMPROMPT
load_dotenv()

client = OpenAI()

class Agent(ABC):
    
    def __init__(self, 
        model: Optional[str] = None, 
        system_prompt: Optional[str] = None
    ) -> None:
        
        model = model if model else os.getenv("OPENAI_DEFAULT_MODEL")
        
        self.memory = []
        self.model = model
        
        if system_prompt:
            self.memory.append({"role": "system", "content": system_prompt})
    
    def add_to_memory(self, role, message):
        self.memory.append({"role": role, "content": message})

    def get_schema(self):
        return self.function
        
    @abstractmethod
    def run(self,prompt):
        """User must define this method. Run the agent"""
    
class ConversationAgent(Agent):
    
    def __init__(self, 
        model: Optional[str] = None, 
        system_prompt: Optional[str] = None,
        name_agent: Optional[str] = None,
        description: Optional[str] = None
    ) -> None:
        super().__init__(model,system_prompt)

        self.name_agent = name_agent
        self.description = description

        self.function={
            "name": self.name_agent,
            "description": self.description,
            "parameters": {
                "type": "object",
                "properties": {
                    "user_prompt": {
                        "type": "string",
                        "description": "User prompt to the agent"
                    }
                },
                "required": ["user_query"]
            }
        }
        
    def run(self,prompt):

        self.add_to_memory("user", prompt)
        completion=''
        stream = client.chat.completions.create(
            messages=self.memory,
            model=self.model,
            stream=True
        )
        print(f"{self.name_agent}:")
        for chunk in stream:
            text_chunk=chunk.choices[0].delta.content
            if text_chunk:
                completion+=text_chunk
                print(text_chunk, end='', flush=True)
        
        self.add_to_memory("assistant", completion)
        
        return completion

In [2]:

class ManagerAgent(Agent):
    
    def __init__(self, 
        model: Optional[str] = None, 
        system_prompt: Optional[str] = None,
        list_agents: Optional[list] = None
    ) -> None:
        super().__init__(model,system_prompt)
        
        self.list_agents=list_agents
    
    
    def get_agent_list_schemas(self):
        return [agent.get_schema() for agent in self.list_agents]   
    
    def run(self,prompt):
        
        print(f"----------------- RUN MANAGER -----------------")
        
        #print(self.memory)
        # User query process
        self.add_to_memory("user", prompt)
        print(f"User: {prompt}")
        
        # Choose what agent i have to execute to resolve the problem
        messages = self.memory + [{"role": "user", "content": f"[PROMPT]\n{prompt}\n\n[AGENTS]\n{self.get_agent_list_schemas()}"}]
        chat_completion = client.chat.completions.create(
            messages=messages,
            model=self.model,
            functions=self.get_agent_list_schemas()
        )
        #print(chat_completion)
                
        # Get the agent to execute
        agent_name=chat_completion.choices[0].message.function_call.name
        agent_prompt=json.loads(chat_completion.choices[0].message.function_call.arguments)['user_prompt']
                
        print(f"Manager: {agent_prompt}")
        self.add_to_memory("user", agent_prompt)
         
        #get the class agent to execute
        class_agent = next((agent for agent in self.list_agents if agent.name_agent == agent_name), None)
        
        #Execute the agent
        for i in range(2):
            response_agent=class_agent.run(agent_prompt)
            
            code_to_execute = extract_code_from_string(response_agent)
            if code_to_execute:
                print('\nManager: Executing code...')
                try:
                    print(f"TEST:\n{code_to_execute}")
                    execute_code(code_to_execute,use_docker=False)
                    print('f"Manager: End execution')
                    break
                except Exception as e:
                    print(f"Error: {e}")
                    agent_prompt=f'{agent_prompt}\n\nError: {e}'
            else:
                print(f"Manager: No code found to execute.")
        
        
        #print(f"{agent_name}: {response_agent}")
        
        self.add_to_memory("user", response_agent)
        
        return response_agent

In [3]:
CodeDeveloper=ConversationAgent(
    model="gpt-4-1106-preview",
    name_agent="CodeDeveloper",
    description="Agent to create code in python",
    system_prompt=CODE_DEVELOPER_SYSTEMPROMPT
)

UIDesignDeveloper=ConversationAgent(
    model="gpt-4-1106-preview",
    name_agent="UIDesignDeveloper",
    description="Agent to resolve UI design problems in python",
    system_prompt=EXECUTE_UIDESIGN_SYSTEMPROMPT
)

DocumentationDeveloper=ConversationAgent(
    model="gpt-4-1106-preview",
    name_agent="DocumentationDeveloper",
    description="Agent to document code",
    system_prompt=DOCUMENTATION_DEVELOPER_SYSTEMPROMPT
)

# CodeExecuter=CodeExecuterAgent()

# Manager Agents
manager=ManagerAgent(
    model="gpt-4-1106-preview",
    list_agents=[
        CodeDeveloper,
        DocumentationDeveloper
    ],
    system_prompt="You need to choose an agent to resolve your development problem and can execute code. You use another agent to generate code, visual design, documentation and others. Not reduce the user query, you must expand it"
)

In [4]:

manager.run("Create a code for a face detector using my camera, show me it in display and it must run by 30 sec and end.")

-----------------ITERATION-----------------
[{'role': 'system', 'content': 'You need to choose an agent to resolve your development problem and can execute code. You use another agent to generate code, visual design, documentation and others. Not reduce the user query, you must expand it'}]
User: Create a code for a face detector using my camera, show me it in display and it must run by 30 sec and end.
ChatCompletion(id='chatcmpl-8QMGHTIHgnoUbFAgyrRt3lheR1rGq', choices=[Choice(finish_reason='function_call', index=0, message=ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{"user_prompt":"I need a Python script that uses OpenCV to detect faces from the webcam feed. The program should display the live video feed with face detection, and terminate automatically after running for 30 seconds."}', name='CodeDeveloper'), tool_calls=None))], created=1701292793, model='gpt-4-1106-preview', object='chat.completion', system_fingerprint='fp_a24b4d720c', u

SIGALRM is not supported on Windows. No timeout will be enforced.



Manager: Executing code...
TEST:
import cv2
import time

# Start the timer
start_time = time.time()

# Load the cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Access the webcam (use 0 for default webcam)
cap = cv2.VideoCapture(0)

# Check if the webcam is opened successfully
if not cap.isOpened():
    raise IOError("Cannot open webcam")

while True:
    # Read the frame from the webcam
    ret, frame = cap.read()
    
    # If frame is read correctly ret is True
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break
    
    # Convert to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect the faces
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)

    # Draw the rectangle around each face
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)

    # Display the output
    cv2.imshow('Face Detect

'```python\nimport cv2\nimport time\n\n# Start the timer\nstart_time = time.time()\n\n# Load the cascade for face detection\nface_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + \'haarcascade_frontalface_default.xml\')\n\n# Access the webcam (use 0 for default webcam)\ncap = cv2.VideoCapture(0)\n\n# Check if the webcam is opened successfully\nif not cap.isOpened():\n    raise IOError("Cannot open webcam")\n\nwhile True:\n    # Read the frame from the webcam\n    ret, frame = cap.read()\n    \n    # If frame is read correctly ret is True\n    if not ret:\n        print("Can\'t receive frame (stream end?). Exiting ...")\n        break\n    \n    # Convert to grayscale\n    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)\n\n    # Detect the faces\n    faces = face_cascade.detectMultiScale(gray, 1.1, 4)\n\n    # Draw the rectangle around each face\n    for (x, y, w, h) in faces:\n        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)\n\n    # Display the output\n    cv2.im

In [5]:
manager.run("I need to develop an additional functionality for the existing code, remove the stop based on time condition. It necessitates the integration of a 'Stop' button which, when activated, will cease the camera operation and visual output. Additionally, the graphic interface must have the features in one specific window.")

-----------------ITERATION-----------------
[{'role': 'system', 'content': 'You need to choose an agent to resolve your development problem and can execute code. You use another agent to generate code, visual design, documentation and others. Not reduce the user query, you must expand it'}, {'role': 'user', 'content': 'Create a code for a face detector using my camera, show me it in display and it must run by 30 sec and end.'}, {'role': 'user', 'content': 'I need a Python script that uses OpenCV to detect faces from the webcam feed. The program should display the live video feed with face detection, and terminate automatically after running for 30 seconds.'}, {'role': 'user', 'content': '```python\nimport cv2\nimport time\n\n# Start the timer\nstart_time = time.time()\n\n# Load the cascade for face detection\nface_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + \'haarcascade_frontalface_default.xml\')\n\n# Access the webcam (use 0 for default webcam)\ncap = cv2.VideoCapture(0)\

SIGALRM is not supported on Windows. No timeout will be enforced.



Manager: Executing code...
TEST:
import cv2
from tkinter import *
from tkinter import ttk
from PIL import Image, ImageTk
import threading

# Function to handle the face detection and video feed
def video_stream():
    ret, frame = cap.read()
    if ret:
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.1, 4)
        for (x, y, w, h) in faces:
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGBA)
        img = Image.fromarray(cv2image)
        imgtk = ImageTk.PhotoImage(image=img)
        label.imgtk = imgtk
        label.configure(image=imgtk)
        label.after(10, video_stream)

# Function to stop the video feed
def stop():
    threading.Thread(target=stop_thread).start()

# Function running in a separate thread to release the webcam and close windows
def stop_thread():
    global running
    running = False
    cap.release()
    cv2.destroyAllWindows

'```python\nimport cv2\nfrom tkinter import *\nfrom tkinter import ttk\nfrom PIL import Image, ImageTk\nimport threading\n\n# Function to handle the face detection and video feed\ndef video_stream():\n    ret, frame = cap.read()\n    if ret:\n        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)\n        faces = face_cascade.detectMultiScale(gray, 1.1, 4)\n        for (x, y, w, h) in faces:\n            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)\n        cv2image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGBA)\n        img = Image.fromarray(cv2image)\n        imgtk = ImageTk.PhotoImage(image=img)\n        label.imgtk = imgtk\n        label.configure(image=imgtk)\n        label.after(10, video_stream)\n\n# Function to stop the video feed\ndef stop():\n    threading.Thread(target=stop_thread).start()\n\n# Function running in a separate thread to release the webcam and close windows\ndef stop_thread():\n    global running\n    running = False\n    cap.release()\n    cv2.destroyAl

In [6]:

manager.run("All code before, document the code and explain it")

-----------------ITERATION-----------------
[{'role': 'system', 'content': 'You need to choose an agent to resolve your development problem and can execute code. You use another agent to generate code, visual design, documentation and others. Not reduce the user query, you must expand it'}, {'role': 'user', 'content': 'Create a code for a face detector using my camera, show me it in display and it must run by 30 sec and end.'}, {'role': 'user', 'content': 'I need a Python script that uses OpenCV to detect faces from the webcam feed. The program should display the live video feed with face detection, and terminate automatically after running for 30 seconds.'}, {'role': 'user', 'content': '```python\nimport cv2\nimport time\n\n# Start the timer\nstart_time = time.time()\n\n# Load the cascade for face detection\nface_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + \'haarcascade_frontalface_default.xml\')\n\n# Access the webcam (use 0 for default webcam)\ncap = cv2.VideoCapture(0)\

"# Python Script Documentation\n\n## Title\n\nReal-time Face Detection in Webcam Feed with Tkinter Control\n\n## Description\n\nThis Python script utilizes the OpenCV library to perform real-time face detection in a webcam video feed. The graphical user interface is built using Tkinter, which includes a 'Stop' button that allows the user to terminate the webcam feed and exit the program.\n\n## Libraries Required\n\n- OpenCV (cv2): An open-source computer vision and machine learning software library. It is used for capturing video from the webcam and processing the frames for face detection.\n- Tkinter: The standard GUI library for Python, used for creating the application window with user controls.\n\n## Components\n\n### Webcam Initialization\n\nThe script begins by initializing the webcam, setting up the necessary properties to capture the video stream.\n\n### Tkinter Window\n\nA GUI window is created using Tkinter that serves as the main interface for user interaction. The window in